<a href="https://colab.research.google.com/github/cdedow/FWE458_2025/blob/main/Homework_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

filedir = '/content/drive/MyDrive/FWE458/ColabNotebooks/'

Mounted at /content/drive


In [2]:
import pandas as pd #importing the pandas library

fname = filedir + "Wcr_GPPdaily.csv" #preparing to load into data frame

df = pd.read_csv(fname) #loaded the file into a pandas data frame
df.head() #displayed the first 5 rows

,TIMESTAMP,TA_F,SW_IN_F,VPD_F,GPP_NT_VUT_REF
0,19990101,-20.063,72.603,0.413,-0.517364
1,19990102,-12.814,12.358,0.147,-0.094241
2,19990103,-12.625,33.132,0.128,-0.166819
3,19990104,-18.652,93.481,0.263,-0.582301
4,19990105,-20.269,45.502,0.261,-0.568240


In [3]:
#data processing
df = df.rename(columns={'TA_F': 'TA'}) #renaming column
df = df.rename(columns={'SW_IN_F': 'SW'}) #renaming column
df = df.rename(columns={'VPD_F': 'VPD'}) #renaming column
df = df.rename(columns={'GPP_NT_VUT_REF': 'GPP'}) #renaming column
df.drop(columns= ['TIMESTAMP'], inplace = True) #dropping the timestamp column
df.dropna(inplace = True) #also dropping any columns that do not have a value (None/NA)
df.head() #returning part of the data frame

,TA,SW,VPD,GPP
0,-20.063,72.603,0.413,-0.517364
1,-12.814,12.358,0.147,-0.094241
2,-12.625,33.132,0.128,-0.166819
3,-18.652,93.481,0.263,-0.582301
4,-20.269,45.502,0.261,-0.568240


In [4]:
df['SW_VPD'] = df['SW'] * df['VPD'] #creating a new column that multiplies the two values from each of the columns row by row
df['SW_TA'] = df['SW'] * df['TA'] #creating a new column that multiplies the two values from each of the columns row by row
df['VPD_TA'] = df['VPD'] * df['TA'] #creating a new column that multiplies the two values from each of the columns row by row
df.head() #returning the first part of the data frame

,TA,SW,VPD,GPP,SW_VPD,SW_TA,VPD_TA
0,-20.063,72.603,0.413,-0.517364,29.985039,-1456.633989,-8.286019
1,-12.814,12.358,0.147,-0.094241,1.816626,-158.355412,-1.883658
2,-12.625,33.132,0.128,-0.166819,4.240896,-418.291500,-1.616000
3,-18.652,93.481,0.263,-0.582301,24.585503,-1743.607612,-4.905476
4,-20.269,45.502,0.261,-0.568240,11.876022,-922.280038,-5.290209


In [5]:
from scipy.optimize import minimize #importing necessary packages
import numpy as np #importing necessary packages

def linmodel(param, SW, GPP, TA, VPD, SW_VPD, SW_TA, VPD_TA): #started by defining the linear model including the parameters and all of the terms to be considered
  b0, b1, b2, b3, b4, b5, b6 = param #assigned the coefficients to be equal to the param temr in the model
  GPPpred = b0 + b1*SW + b2 * VPD + b3 * TA + b4 * (SW_VPD) + b5 * (SW_TA) + b6 * (VPD_TA) #put in the prediction equation
  mse = (np.mean((GPPpred - GPP)**2)) #goes row by row calculating the mean squared error by taking the predicted value based on the values in that row and substracts it from the actual value (the error) squares that value (to eliminate negative values) and takes the mean of all of those values
  return mse #returns the means squared errors


In [6]:
# minimizing coefficients
SW = df['SW'] #creating variables that contain the values in the corresponding column to make it easier to access
GPP = df['GPP'] #creating variables that contain the values in the corresponding column to make it easier to access
TA = df['TA'] #creating variables that contain the values in the corresponding column to make it easier to access
VPD = df['VPD'] #creating variables that contain the values in the corresponding column to make it easier to access
SW_VPD = df['SW_VPD'] #creating variables that contain the values in the corresponding column to make it easier to access
SW_TA = df['SW_TA'] #creating variables that contain the values in the corresponding column to make it easier to access
VPD_TA = df['VPD_TA'] #creating variables that contain the values in the corresponding column to make it easier to access

guess = [0,0,0,0,0,0,0] #providing and intial guess as the baseline for the model

min = minimize(linmodel, guess, args = (SW, GPP, TA, VPD, SW_VPD, SW_TA, VPD_TA)) #assigned a variable to the values that are produced by the linear model defined above
print(min) #printed out what was contained in the variable

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 15035851.453687316
        x: [-9.446e+02 -3.246e+00 -6.484e+02  8.215e+01  1.792e+00
            -4.560e-01  1.360e+01]
      nit: 17
      jac: [ 0.000e+00  2.500e-01  0.000e+00  0.000e+00 -1.500e+00
             6.250e-01 -1.250e-01]
 hess_inv: [[ 8.022e-03 -7.076e-05 ...  3.186e-06 -7.774e-04]
            [-7.076e-05  8.498e-05 ... -9.615e-07  2.009e-04]
            ...
            [ 3.186e-06 -9.615e-07 ...  2.827e-07 -2.713e-06]
            [-7.774e-04  2.009e-04 ... -2.713e-06  1.576e-03]]
     nfev: 336
     njev: 42


In [7]:
#linear regression
from sklearn.linear_model import LinearRegression #importing sklearn to complete linear regression

x = df[['SW', 'VPD', 'TA', 'SW_VPD', 'SW_TA', 'VPD_TA']] #defining the independent variables
y = df['GPP'] #defining the target variable

model = LinearRegression() #assigning the variable to the linear regression model

model.fit(x, y) #then running the linear regression model best fit to the previously defined x and y variables

print("Coefficients: ", model.coef_) #printed out the coefficients from the model
print("Intercept: ", model.intercept_) #printed out the intercept value from the model

Coefficients:  [-3.24914968e+00 -6.48574173e+02  8.21699813e+01  1.79249151e+00
 -4.56050357e-01  1.35954025e+01]
Intercept:  -944.0672018203164


In [8]:
#scipy r squared

ones = np.ones((x.shape[0], 1)) #i had to add an intercept column to my x values because the function wouldn't work without it
x_with_intercept = np.hstack([ones, x]) #this is making a new value for x which includes all of the old ones and added the intercept column

beta = min.x #established that beta equals the minimum of x

def predict(beta, x): #i chose to make a function that would compute the dot product between x and beta
    return np.dot(x, beta) #returns the dot product

y_pred_scipy = predict(beta, x_with_intercept) #i used the function above with the beta variable and the new x series i made

residuals = np.sum((y - y_pred_scipy) ** 2) #this calculates the sum of squared residuals
total_sum = np.sum((y - np.mean(y)) ** 2) #and this computes the total sum of y minus the mean of y and that squared (similar to standard deviation)

r_squared_scipy = 1 - (residuals / total_sum) #finishes computing the rest of r squared
print("R-squared (SciPy):", r_squared_scipy) #prints out the r squared value for the model

R-squared (SciPy): 0.01358269810244761


In [10]:
#sklearn r squared

y_pred_sklearn = model.predict(x) #used the predict function on the x variable that was defined above
sk_residuals = np.sum((y - y_pred_sklearn) ** 2) #computes the differenece between the actual value of y and the predicted value, squares it, and sums all of those values
sk_total_sum = np.sum((y - np.mean(y)) ** 2) #computes the differenece between the actual value of y and the mean value of y, squares it, and sums all of those values
r_squared_sklearn = 1 - (sk_residuals / sk_total_sum) #finishes computing the r squared value
print("R-squared (Sklearn):", r_squared_sklearn)

R-squared (Sklearn): 0.013582700933254976
